In [164]:
import pandas as pd
import pyam
from message_ix import make_df

MODEL = 'MESSAGEix'
SCEN = 'none'

cols = ['region', 'technology', 'year', 'value', 'unit']

ppl = pd.read_excel('historical_activity_WIP.xlsx',
                    sheet_name = 'powerplants',
                    usecols = cols,
                   ).dropna()

therm = pd.read_excel('historical_activity_WIP.xlsx',
                      sheet_name = 'thermal',
                      usecols = cols,
                     ).dropna()

other_elec = pd.read_excel('historical_activity_WIP.xlsx',
                           sheet_name = 'other_electricity',
                           usecols = cols,
                          ).dropna()

prod = pd.read_excel('historical_activity_WIP.xlsx',
                     sheet_name = 'production',
                     usecols = cols,
                    ).dropna()

# Concatenate data
total = pd.concat([ppl, therm, other_elec, prod],
                  axis = 'rows').set_index('region').sort_index()

# Create IAM df
pyam_total = pyam.IamDataFrame(total,
                               model = MODEL,
                               scenario = SCEN,
                               variable = 'technology')

# Unit conversion
pyam_total.convert_unit('PJ', to = 'GWa', inplace = True)
pyam_total.rename(unit = {'GWa': 'Gwa'}, inplace = True)

## Downscaling
From Stat Can data

# Downscale data with only Canadian values
can_vars = [x for x in pyam_total.filter(region = 'Canada').variable if x not in pyam_total.filter(region = 'Ontario').variable]
can_vars.sort()

# Create dictionary to categorize variables
vars_dict = {'coal': [x for x in can_vars if x.split('_')[0] == 'coal'],
             'gas': [x for x in can_vars if x.split('_')[0] == 'gas'],
             'oil': [x for x in can_vars if x.split('_')[0] == 'oil']}

# Downscale from Stat Can national data

for value in vars_dict['coal']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'coal_ppl', append = True)
        
for value in vars_dict['gas']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'gas_ppl', append = True)
    
for value in vars_dict['oil']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'foil_ppl', append = True)

## Downscaling
From National MESSAGE model

# Read in Canadian data
data = pd.read_excel('MESSAGE_CA__test__v4.xlsx',
                         sheet_name = 'historical_activity',
                         usecols = ['node_loc', 'technology', 'year_act',
                                    'value', 'unit'])

# Sort to only include year = 2015, non-zero values
msg_can = data[(data['year_act']  == 2015) &
                    (data['value'] != 0)].reset_index(drop = True)

total_vars = msg_can['technology'].values

# Filter to only include missing variables
downscale_vars = [x for x in msg_can['technology'].unique() if x not in pyam_total.variable]
msg_can = msg_can[msg_can['technology'].isin(downscale_vars)]

# Create IAM df
pyam_msg_can = pyam.IamDataFrame(msg_can,
                                 model = MODEL,
                                 scenario = SCEN,
                                 region = 'node_loc',
                                 variable = 'technology',
                                 year = 'year_act')

pyam_msg_can.rename(unit = {'GWa': 'Gwa'}, inplace = True)

# Read in gdp data for downscaling
gdp = pd.read_excel('/home/noah/Documents/Co-op/messageDataTesting/actBounds/pop_gdp_proj.xlsx',
                    sheet_name = 'gdp_pop_proj')

gdp = gdp[(gdp['variable'] == 'GDP (PPP)') & 
          (gdp['year'] == 2015)].reset_index(drop = True)

pyam_gdp = pyam.IamDataFrame(gdp,
                             model = MODEL,
                             scenario = SCEN)

pyam_total.append(pyam_msg_can, inplace = True)
pyam_total.append(pyam_gdp, inplace = True)

# Create dictionary to categorize variables
biomass = [x for x in downscale_vars if x.split('_')[0] == 'biomass']
coal = [x for x in downscale_vars if x.split('_')[0] == 'coal']
elec = [x for x in downscale_vars if x.split('_')[0] == 'elec']
foil = [x for x in downscale_vars if x.split('_')[0] == 'foil']
gas = [x for x in downscale_vars if x.split('_')[0] == 'gas']
loil = [x for x in downscale_vars if x.split('_')[0] == 'loil']
other = [x for x in downscale_vars if x not in set(biomass + coal + elec + foil + gas + loil)]


downscale_dict = {'biomass': biomass,
                  'coal': coal,
                  'elec': elec,
                  'foil': foil,
                  'gas': gas,
                  'loil': loil,
                  'other': other}

for value in downscale_dict['biomass']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'bio_ppl', append = True)
    
for value in downscale_dict['coal']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'coal_ppl', append = True)
    
for value in downscale_dict['elec']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'elec_exp', append = True)

for value in downscale_dict['foil']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'foil_ppl', append = True)

for value in downscale_dict['gas']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'gas_ppl', append = True)

for value in downscale_dict['loil']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'loil_ppl', append = True)

for value in downscale_dict['other']:
    pyam_total.downscale_region(value, region = 'Canada', proxy = 'GDP (PPP)', append = True)

final = pyam_total.filter(variable = total_vars)
final.to_csv('historical_activity.csv')